First roslaunch without skill-server.

Then run 
```bash
rosrun hrr_cobot_robot skill_server --no-compile-nb __ns:="hrr_cobot"
```
<!-- 
```bash
rosnode kill /hrr_cobot/skill_server
```

For testing run

```bash
rosrun hrr_cobot_robot skill_server --no-compile-nb __ns:="hrr_cobot"
```

before executing the code within this notebook. For additional help and instructions run

```bash
rosrun hrr_cobot_robot skill_server --help
``` -->

<!-- ### Debugging mode

For debugging, use an IDE, e.g. [pycharm](https://www.jetbrains.com/pycharm/download/) or vscode(https://code.visualstudio.com/) with proper plugins, and run the ``skill_server`` script from ``hrr_cobot_robot->scripts->skill_server`` in debug mode, set breakpoints as needed and call the action-clients from here.

You may want to add the proper namespace to emulate identical beahvior, thus adjust the settings

![access-settings](./media/pycharm_skill_server_access_settings.png)

and add the namespace as a Parameter

![set namespace](./media/pycharm_skill_server.png) -->

<!-- ## set up basics

For the code below, we assume that the rqt graph shows something like this

![rqt-graph](./media/skill_server_rqt_graph.png)

**NOTE: this graph is missing some actions and still uses an Arduino interface, but serves as a reference** -->

In [1]:
# math and robotics
import pathlib
import numpy as np
import spatialmath as sm
# ROS
import actionlib
import rospy
import hrr_common
from hr_recycler_msgs.msg import (
    AdaptiveGraspingAction, AdaptiveGraspingGoal,
    CuttingAction, CuttingGoal,
    ChangeToolAction, ChangeToolGoal,    
    FingerGraspAction, FingerGraspGoal,    
    PushOpenAction, PushOpenGoal,
    VacuumGraspingAction, VacuumGraspingGoal,
    UnscrewAction, UnscrewGoal
)
from hrr_msgs.msg import CalibrateCobotAction, CalibrateCobotGoal
# set printing and plotting options
np.set_printoptions(precision=5, suppress=True)
from hr_recycler_msgs.msg import CuttingAction, CuttingFeedback, CuttingResult, CuttingGoal, SkillResult, CobotState, MaterialType
import actionlib
from hr_recycler_msgs.msg import ToolType
from hr_recycler_msgs.msg import PlannerAction , PlannerGoal
import hrr_common as hrr_cm
from comau_msgs.srv import SetIO, SetIORequest
rospy.init_node('TUM_notebook')
cobot_ns = hrr_common.ros_utils.fix_prefix(hrr_common.ros_utils.get_param("/cobot_ns"))
change_tool_action_srv_name = hrr_common.ros_utils.get_param(f"{cobot_ns}change_tool_action_srv_name")
calibration_action_srv_name = hrr_common.ros_utils.get_param(f"{cobot_ns}calibration_action_srv_name")
cutting_action_srv_name = hrr_common.ros_utils.get_param(f"{cobot_ns}cutting_action_srv_name")
grinding_action_srv_name = hrr_common.ros_utils.get_param(f"{cobot_ns}grinding_action_srv_name")
pc_opening_action_srv_name = hrr_common.ros_utils.get_param(f"{cobot_ns}pc_opening_action_srv_name")
sensitive_grasping_action_srv_name = hrr_common.ros_utils.get_param(f"{cobot_ns}sensitive_grasping_action_srv_name")
unscrew_action_srv_name = hrr_common.ros_utils.get_param(f"{cobot_ns}unscrew_action_srv_name")
vacuum_pick_place_action_srv_name = hrr_common.ros_utils.get_param(f"{cobot_ns}vacuum_pick_place_action_srv_name")
finger_grasping_action_srv_name= hrr_common.ros_utils.get_param(f"{cobot_ns}finger_grasping_action_srv_name")
def get_client(action_topic, ActionClass, timeout_secs=2.0):
    client = actionlib.SimpleActionClient(action_topic, ActionClass)
    client.wait_for_server(timeout=rospy.Duration(timeout_secs))
    return client
def get_result(cl):
    cl.wait_for_result()
    return cl.get_result()
def cancel(cl):
    cl.cancel_all_goals()
import hrr_cobot_robot as hrr_rob
cobot = hrr_rob.HrrCobotControl.from_ros("/hrr_cobot")

         at line 278 in /tmp/binarydeb/ros-noetic-tf2-0.7.5/src/buffer_core.cpp


[rosout] [/TUM_notebook] Initialize MATLAB-Motion planner @ /home/hrr_cobot/_ros/hr_recycler_ws/src/planner/src
[rosout] [/TUM_notebook] connected to Matlab
Shutting down global node /matlab_global_node_42785 with NodeURI http://192.168.2.87:45243/
The value of the ROS_IP environment variable, 192.168.2.87, will be used to set the advertised address for the ROS node.
Initializing global node /matlab_global_node_99566 with NodeURI http://192.168.2.87:37929/
[rosout] [/TUM_notebook] MATLAB-Initialization done


pybullet build time: May  8 2021 05:48:13


[rosout] some functions are not yet compiled. Expect delays upon first call


### Check FT sensor

In [ ]:
#Check if force values are not crazily high (above 40 or 50)
cobot.B_F_msr #gives back force in x y z and torque around x y z axis

In [ ]:
cobot.FT.reset_bias()

### Correct tool set?
Rosparam needs to be correct before skill-server is launched. Otherwise cobot inside skill-server has the wrong tool. cobot.change_tool() in this notebook does not affect skill-server cobot. Procedure: kill skill server, change tool, then restart skill server

In [ ]:
!rosparam get /hrr_cobot/tool_name

In [2]:
#cobot.change_tool("nothing")
#cobot.change_tool("screwdriver")
#cobot.change_tool("wsg_50_dsa")
cobot.change_tool("wsg_50") #Pneumatic Finger Gripper
#cobot.change_tool("shaftgrinder")
# cobot.change_tool("vacuum")

True

To do:
DOnt send cancel skill if something goes wrong
wsg50 connection unstable, check connectors with pietro 
check big cable with pietro, almost goes underneath EE when pickung up tools
Vacuum for PSU go towards middle, take normal vector as orientation
Approach surface in unscrewing does not work reliably

### Cancel Skills
This also works if task planner calls them

In [8]:
shaftgrinder_client=get_client(cutting_action_srv_name, CuttingAction)
unscrewing_client = get_client(unscrew_action_srv_name, UnscrewAction)
vacuum_client = get_client(vacuum_pick_place_action_srv_name, VacuumGraspingAction)
pgrip_client = get_client(finger_grasping_action_srv_name, FingerGraspAction)
tool_change_client = get_client(change_tool_action_srv_name, ChangeToolAction)
grasping_client = get_client(sensitive_grasping_action_srv_name, AdaptiveGraspingAction)
calibrate_client = get_client(calibration_action_srv_name, CalibrateCobotAction)

In [37]:
cancel(shaftgrinder_client)
cancel(unscrewing_client)
cancel(tool_change_client)
cancel(vacuum_client)
cancel(pgrip_client)
cancel(grasping_client)

In [ ]:
cobot.T_E_C_robot

### Moving the robot

In [ ]:
cartesian_difference = [-0.1, -0.1,0.0]
cobot.goTo(sm.SE3(cartesian_difference)@cobot.T_B_E_robot,v_max=0.05)

In [ ]:
cartesian_difference = [-0.0, -0.,0.2]
cobot.goTo(sm.SE3(cartesian_difference)@cobot.T_B_E_robot,v_max=0.05)

In [34]:
q_neutral = np.r_[1.34109, 0.09328, -1.44435, -0.01627, 0.67686, -0.00009]
cobot.move_to_joint_pose(q_neutral, stochastic=False)

[rosout] Moving linearly if planner didn't move. Take care!


In [ ]:
#PC Tower standing unscrewing pose
cobot.move_to_joint_pose(np.r_[ 0.96828,  0.17584, -1.5306 , -1.55063,  1.62805, -3.01406], stochastic=False)

In [ ]:
cobot.move_to_joint_pose(cobot.q_calib, stochastic=False)

### Open close Tool changer

In [ ]:
cobot.B_F_msr

In [ ]:
cobot.FT.f

In [ ]:
cobot.FT.reset_bias()

In [ ]:
cobot.close_tool_changer()

In [38]:
cobot.open_tool_changer()

In [ ]:
print(cobot)

### Open clamping
The clamp will open (release air) if cobot pin 300 is set to True. Need be set to False after, so that someone else can close it again.

In [ ]:
from comau_msgs.srv import SetIO, SetIORequest
srv = rospy.ServiceProxy("/hrr_cobot/set_digital_io", SetIO)
srv(SetIORequest(pin=300, state=True)) 
rospy.sleep(1.0)
srv(SetIORequest(pin=300, state=False))

## Test tools

### Gripper

In [ ]:
cobot.change_tool("nothing")

In [ ]:
cobot.change_tool("wsg_50_dsa")

In [ ]:
cobot.gripper.width

In [ ]:
cobot.gripper.reset()

In [ ]:
cobot.gripper.width

In [ ]:
cobot.gripper.reset()
rospy.sleep(0.5)
cobot.gripper.send_pos(0.07, si=True)

In [ ]:
cobot.gripper.send_pos(0.07, si=True)

In [ ]:
cobot.change_tool("nothing")

In [ ]:
cobot.change_tool("wsg_50_dsa")

In [ ]:
#Test
cobot.gripper.reset()
rospy.sleep(3.0)
oldwidth = np.copy(cobot.gripper.width)
print(cobot.gripper.width)
cobot.gripper.set_vel(-0.1,si=True)
rospy.sleep(0.5)
cobot.gripper.set_vel(0.0,si=True)
print(cobot.gripper.width)
print(oldwidth==cobot.gripper.width)

In [ ]:
cobot.gripper.set_vel(0,si=True)

In [ ]:
cobot.gripper.send_pos(0.06, si=True)

In [ ]:
cobot.gripper.set_vel(-0.015,si=True)
while cobot.gripper.width>0.008:
    pass
cobot.gripper.set_vel(0.0, si=True)

### Vacuum

In [ ]:
cobot.change_tool("vacuum")

In [ ]:
cobot.tool_controller.vacuum = True

In [ ]:
cobot.tool_controller.vacuum = False

### Shaftgrinder

In [ ]:
cobot.change_tool("shaftgrinder")

In [ ]:
cobot.tool_controller.run_shaftgrinder(0.1, "slow")

### Screwdriver

In [ ]:
cobot.change_tool("screwdriver")

In [ ]:
cobot.run_screwdriver_program(1, run_time=2)

# Send skill goals ourselves

## Cutting

In [ ]:
material= MaterialType()
material = material.PLASTIC
normalA = hrr_cm.ros_utils.np2vector3stamped(np.r_[0,0,1], "normalA")
pointA = hrr_cm.ros_utils.np2vector3stamped(np.r_[0.755,-0.209,0.23], "pointA")#[0.6921,0.2472,0.], "pointA")
normalB = hrr_cm.ros_utils.np2vector3stamped(np.r_[0,0,1], "normalB")
pointB = hrr_cm.ros_utils.np2vector3stamped(np.r_[0.755,-0.1898,0.23], "pointB")
goal=CuttingGoal()
goal.surface_normal_start=normalA
goal.surface_normal_end=normalB
goal.start_location=pointA
goal.end_location=pointB
goal.material.material_type=material
goal.timeout=5000
goal.thickness = 0.005 #not important. can be anything

In [ ]:
shaftgrinder_client.send_goal(goal)

In [ ]:
cancel(shaftgrinder_client)

In [ ]:
get_result(shaftgrinder_client)

## Calibration

In [ ]:
goal = CalibrateCobotGoal()
data_path = pathlib.Path.home() / "Documents" / "skill_recordings"
data_path.mkdir(parents=True, exist_ok=True)
tool_name = hrr_common.ros_utils.get_param(f'{cobot_ns}tool_name')
if tool_name == '':
    tool_name = 'nothing'
goal.calibration_file = str(data_path / f"{tool_name}.pkl")
goal.data_file = str(data_path / f"{tool_name}_calibration.npy")
goal.recalibrate = True
goal.scale_q4 = 0.05
goal.scale_q5 = 0.05
goal.scale_q6 = 0.05
goal.q4_dot_des = 0.02
goal.q5_dot_des = 0.02
goal.q6_dot_des = 0.02
goal.keep_current_pose=True
goal

In [ ]:
calibrate_client.send_goal(goal)

In [ ]:
cancel(calibrate_client)

In [ ]:
# get client result / feedback
get_result(calibrate_client)

In [ ]:
# plot results -> copy line to terminal
print(f"please run the following line in a separate terminal:\nrosrun hrr_cobot_robot plot_ft_calibration -p {goal.calibration_file} {goal.data_file}")

In [ ]:
cobot.update()
cobot.B_F_msr, cobot.FT_F, cobot.FT.f,cobot.FT.wrench_calib, cobot.FT.wrench, cobot.FT.offset

In [ ]:
cobot.FT_F

In [ ]:
cobot.B_F_msr

In [ ]:
#cobot.FT.read_params()

In [ ]:
#cobot.update_tf()

In [ ]:
cobot.R_FT_E

In [ ]:
cobot.FT.offset

In [ ]:
cobot.FT.reset_bias() 

In [ ]:
cobot.change_tool("vacuum")

In [ ]:
cobot.tool_controller.vacuum = False

## Unscrewing 

In [ ]:
#Measure a screw manually by inserting tip of screwdriver and executing this cell
H = cobot.T_B_C_robot.t
H

In [ ]:
unscrewing_client = get_client(unscrew_action_srv_name, UnscrewAction)

In [ ]:
x: 0.509153496943
y: 0.000925782585431
z: 0.420499908771

In [ ]:
#For tilted microwave
B_normal = sm.base.rotx(10, "deg") @ sm.base.unitvec(np.r_[0., 0.0, 1.0])
B_screw_guess = H #np.r_[0.55, -0.145, 0.24531]

In [ ]:
#Screw 1 perfect
B_normal = np.r_[0,0,1]
B_screw_guess = np.r_[0.5055, 0.0022, 0.4189]#H 

In [28]:
#Screw 1 wrong
B_normal = np.r_[0,0,1]
B_screw_guess = np.r_[0.636, 0.093, 0.44426]#H 

In [29]:
sc_goal = UnscrewGoal()
sc_goal.screw_location.header.frame_id = "hrr_cobot.base_link"
sc_goal.surface_normal.header.frame_id = "hrr_cobot.base_link"
sc_goal.surface_normal.vector = hrr_common.ros_utils.np2vec3(B_normal)
sc_goal.screw_location.point = hrr_common.ros_utils.np2vec3(B_screw_guess)
sc_goal.contact_force = 5
sc_goal.insertion_force = 10
sc_goal.timeout = 1200.0
# sc_goal

In [30]:
unscrewing_client.send_goal(sc_goal)

In [ ]:
cancel(unscrewing_client)

## Vacuum Grasping

In [ ]:
cobot.tool

In [ ]:
obj_pos_guess = np.r_[0.6308, 0.1, 0.1044] #[0.66, 0.17, 0.07] is actual vision data for elamp cover
obj_orient_guess = np.quaternion( 0.0775352159744,0.355686284944, 0.519448327377, 0.773077610704)#np.quaternion(w x y z)

release_pos = np.r_[0, 0.56, 0.20] # will drop the object here, careful!
release_orientation = obj_orient_guess

In [ ]:
vac_goal = VacuumGraspingGoal()
vac_goal.object_center.header.frame_id = "hrr_cobot.base_link"
vac_goal.object_center.pose.position = hrr_common.ros_utils.np2vec3(obj_pos_guess)
vac_goal.object_center.pose.orientation = hrr_common.ros_utils.np2quat(obj_orient_guess)
vac_goal.release_pose.header.frame_id = "hrr_cobot.base_link"
vac_goal.release_pose.pose.position = hrr_common.ros_utils.np2vec3(release_pos)
vac_goal.release_pose.pose.orientation = hrr_common.ros_utils.np2quat(release_orientation)

In [ ]:
vacuum_client.send_goal(vac_goal)

In [ ]:
cancel(vacuum_client)

## Pneumatic Finger

In [ ]:
cobot.tool

In [ ]:
x: 0.743623173205
      y: 0.00347981750303
      z: 0.024420364201

In [ ]:
# Haptic Detection
obj_pos_guess = np.r_[0.6893, 0.0316,  0.024420364201] #[0.712, 0.263, 0.0196] is actual vision data for battery
obj_orient_guess = np.quaternion(0.81190278623,0,-0.5759846,-0.095160) #actual vision data for battery
release_pos = np.r_[0, 0.5, 0.10] #choose so it makes sense
release_orientation = np.quaternion(1,1,0,0) #same orientaiton for ease

In [ ]:
# Trust Vision
obj_pos_guess = np.r_[0.6853, 0.0197,  0.024420364201] #[0.712, 0.263, 0.0196] is actual vision data for battery
obj_orient_guess = np.quaternion(0.81190278623,0,-0.5759846,-0.095160) #actual vision data for battery
release_pos = np.r_[0, 0.5, 0.10] #choose so it makes sense
release_orientation = np.quaternion(1,0,0,0) #same orientaiton for ease

In [ ]:
pgrip_goal = FingerGraspGoal()
pgrip_goal.object_center.header.frame_id = "hrr_cobot.base_link"
pgrip_goal.object_center.pose.position = hrr_common.ros_utils.np2vec3(obj_pos_guess)
pgrip_goal.object_center.pose.orientation = hrr_common.ros_utils.np2quat(obj_orient_guess)

pgrip_goal.release_pose.header.frame_id = "hrr_cobot.base_link"
pgrip_goal.release_pose.pose.position = hrr_common.ros_utils.np2vec3(release_pos)
pgrip_goal.release_pose.pose.orientation = hrr_common.ros_utils.np2quat(release_orientation)

pgrip_goal.timeout = 10000.0
#pgrip_goal.gripper_open_width = 0.084
#pgrip_goal.gripper_close_width = 0
# print(pgrip_goal)

In [ ]:
pgrip_client.send_goal(pgrip_goal)

In [ ]:
cancel(pgrip_client)

## Tool Change routine

You may check the tool-change via

- rviz and checking the robot tool-chain to be correct
- varifying the ROS-parameter `/hrr_cobot/tool_name`

In [ ]:
cobot.change_tool("wsg_50")

In [ ]:
!rosparam get /hrr_cobot/tool_name

In [ ]:
cobot.tool

In [35]:
tc_goal = ChangeToolGoal()
# set as needed

tc_goal.new_tool.type = tc_goal.new_tool.NONE
# tc_goal.new_tool.type = tc_goal.new_tool.WSG_50 #pneumatic gripper
# tc_goal.new_tool.type = tc_goal.new_tool.WSG_50_DSA
tc_goal.new_tool.type = tc_goal.new_tool.SHAFT_GRINDER
# tc_goal.new_tool.type = tc_goal.new_tool.SCREW_DRIVER
tc_goal.new_tool.type = tc_goal.new_tool.VACUUM_GRIPPER

In [36]:
tool_change_client = get_client(change_tool_action_srv_name, ChangeToolAction)
tool_change_client.send_goal(tc_goal)

In [ ]:
cancel(tool_change_client)

## Adaptive Grasping

<!-- This skill may be the most sensitive as the gripper driver is not really stable against using the tool-changer usage.

In (programming) theory, the gripper-driver is started via

```bash
roslaunch hrr_cobot_robot hrr_wsg_50.launch
```

via the WSG-watchdog, i.e. every time the tool is set to

```bash
rosparam set /hrr_cobot/tool_name "wsg_50_dsa"
```

and the gripper-IP can be pinged, but reality is different, so the rqt-graph below shows the skill-server with all nodes for the sensitive grasping

![rqt-graph](./media/hrr_wsg_setup_rqt.png)

**NOTE: The velocity based variant requires ```do-mpc```, so you need to run
```pip install do-mpc``` to replicate the setup below**

### Robot setup without skill-server

![rqt-graph](./media/rqt_without_skill_server.png) -->

In [ ]:
cobot.change_tool("nothing")

In [ ]:
cobot.change_tool("wsg_50_dsa")

In [ ]:
cobot.gripper.reset()

In [ ]:
cobot.gripper.width

In [ ]:
cobot.T_B_E_robot, cobot.T_B_C_robot

In [ ]:
!rosparam get 

In [ ]:
grasp_goal = AdaptiveGraspingGoal()
T_B_C_test = sm.SE3(0.6505, -0.008, 0.042)
dispose = sm.SE3(0.4, 0.3, 0.2)
grasp_goal.object_center.pose.position = hrr_common.np2vec3(T_B_C_test.t)
grasp_goal.object_center.pose.orientation = hrr_common.np2quat(hrr_common.rotmat2quat(T_B_C_test.R))
grasp_goal.dispose_off_location_pose.pose.position = hrr_common.np2vec3(dispose.t)
grasp_goal.dispose_off_location_pose.pose.orientation = hrr_common.np2quat(hrr_common.rotmat2quat(dispose.R))
grasp_goal.contact_force = 5.0
grasp_goal.timeout = 20000.0

In [ ]:
grasping_client.send_goal(grasp_goal)

In [ ]:
cancel(grasping_client)

In [ ]:
cobot.gripper.reset()

In [ ]:
cobot.gripper.width